<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:

#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [2]:

#!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

**Getting important libraries**

In [4]:

import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from keras.applications import VGG16
from keras.applications.resnet50 import preprocess_input
from keras.applications.vgg16 import preprocess_input

**Global constants**

1. Since we are dealing with two classes,  *num_classes* is set 2. 
2. Given the fact that VGG16 model was built and trained using images of size (224 x 224), it is therefore needed to resize the images from (227 x 227) to (224 x 224).
3. The training and validating of the model will be done using batches of 100 images.


In [5]:
# 2. defining global constants like batch size
num_classes = 2
image_resize = 224

# either training or validation batch_size is 100
batch_size = 100

**Constructing an ImageDataGenerator and Creating the training and validation generator**

In [6]:
# 3. Constructing ImageDataGenerator instances
data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
)

# training generator
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size = (image_resize, image_resize),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 30001 images belonging to 2 classes.


In [7]:
# valication generator
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size = (image_resize, image_resize),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 9501 images belonging to 2 classes.


****Creating a sequential model using Keras and adding VGG16 model to it and dense layer****

In [8]:
# 4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
model = Sequential()

model.add(VGG16(
    include_top = False,
    pooling = 'avg',
    weights = 'imagenet'
))

58892288/58889256 [==============================] - 0s 0us/step


In [9]:
model.add(Dense(num_classes, activation = 'softmax'))

In [10]:
model.layers

In [11]:
model.layers[0].trainable = False

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


 **Compiling the model using the adam optimizer and the categorical_crossentropy loss function**

In [13]:
#5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

**Fitting the model on the augmented data using the ImageDataGenerators**

In [14]:
%%time
#6. Fit the model on the augmented data using the ImageDataGenerators.
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 1

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
301/301 [==============================] - 19716s 66s/step - loss: 0.1304 - accuracy: 0.9628 - val_loss: 0.0356 - val_accuracy: 0.9921
CPU times: user 10h 43min 30s, sys: 2min 6s, total: 10h 45min 36s
Wall time: 5h 29min 28s


In [16]:
model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

**Loading the ResNet Model**

In [17]:
# 1. Load your saved model that was built using the ResNet50 model. 
from keras.models import load_model
resnet = load_model('classifier_resnet_model.h5')

**Constructing an ImageDataGenerator for the test set**

In [18]:
# 2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
data_gen = ImageDataGenerator()
test_gen = data_gen.flow_from_directory(
           'concrete_data_week4/test',
           target_size = (224, 224),
           shuffle = False
)

Found 500 images belonging to 2 classes.


**Evaluating the models on the test data and printing out their perfomances**

In [19]:
# 3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
vgg16 = model.evaluate_generator(test_gen)
print('VGG16')
print(vgg16)
print('Loss : ', str(vgg16[0]))
print('Accuracy : ', str(vgg16[1]))
# 4. Print the performance of the classifier using the VGG16 pre-trained model.

Instructions for updating:
Please use Model.evaluate, which supports generators.
VGG16
[0.2850046455860138, 0.8659999966621399]
Loss :  0.2850046455860138
Accuracy :  0.8659999966621399


In [20]:
# 5. Print the performance of the classifier using the ResNet pre-trained model.
res50 = resnet.evaluate_generator(test_gen)
print('RESNET50')
print(res50)
print('Loss : ', str(res50[0]))
print('Accuracy : ', str(res50[1]))

RESNET50
[0.04935581609606743, 0.9800000190734863]
Loss :  0.04935581609606743
Accuracy :  0.9800000190734863


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

**Checking out predictions**

In [24]:
import numpy as np
y_pred_vgg = model.predict_generator(test_gen)
y_pred_res = resnet.predict_generator(test_gen)

def predict(x_test) :
  for predictions in x_test:
     prediction = np.argmax(predictions)
     if (prediction == 0) :
       print('Negative')
     else :
        print('Positive') 

**VGG16 predictions**

In [27]:
# vgg16 predictions
print('VGG16 predictions of first 5 elements in test')
predict(y_pred_vgg[0:5])

VGG16 predictions of first 5 elements in test
Negative
Negative
Negative
Negative
Negative


**RESNET predictions**

In [28]:
# resnet50 predictions
print('ResNet50 predictions of first 5 elements in test')
predict(y_pred_res[0:5])

ResNet50 predictions of first 5 elements in test
Negative
Negative
Negative
Negative
Negative


**Checking against the true values**

In [29]:
# true values
print('True first 5 elements in test')
predict(test_gen.next()[1][0:5])

True first 5 elements in test
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).